# Flight delays and weather condition – Data Exploration

**Purpose**
- Explore data for flight delays and weather condition use cases
- Validate assumptions before adding API endpoints
- Prototype logic for FastAPI services

**Author:** Rashed  
**Date:** 2025-24-12


In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("robikscube/flight-delay-dataset-20182022")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'flight-delay-dataset-20182022' dataset.
Path to dataset files: /kaggle/input/flight-delay-dataset-20182022


In [66]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [85]:
%cd /content/drive/MyDrive/Colab\ Notebooks/FlightDelay
%ls

/content/drive/MyDrive/Colab Notebooks/supplychainAPI
aeroapi-python/     data/       openmeteo_api/  sublemmentary_folder/
aeromarket_api/     main.py     __pycache__/
bootstrap_paths.py  notebooks/  pyproject.toml


In [3]:
%ls /kaggle/input/flight-delay-dataset-20182022/

Airlines.csv                   Combined_Flights_2021.csv
Combined_Flights_2018.csv      Combined_Flights_2021.parquet
Combined_Flights_2018.parquet  Combined_Flights_2022.csv
Combined_Flights_2019.csv      Combined_Flights_2022.parquet
Combined_Flights_2019.parquet  raw/
Combined_Flights_2020.csv      readme.html
Combined_Flights_2020.parquet  readme.md


In [4]:
import pandas as pd
# We only need few columns for our analysis
KEEP_COLS = [
    # ---- Date & time (join keys) ----
    "FlightDate",
    "DepTimeBlk",

    # ---- Airports (for direction) ----
    "Origin",
    "Dest",

    # ---- Flight identity (API join) ----
    "IATA_Code_Operating_Airline",

    # ---- Targets / outcomes ----
    "DepDelayMinutes",

    # ---- Operational signal ----
    "Distance",
    "CRSElapsedTime",

    # ---- Cancellation info (to drop)----
    "Cancelled",

    # ---- Arrival time to calculate counts ----
    "ArrTimeBlk",

]

df_2018 = pd.read_parquet('/kaggle/input/flight-delay-dataset-20182022/Combined_Flights_2018.parquet', columns=KEEP_COLS)
df_2019 = pd.read_parquet('/kaggle/input/flight-delay-dataset-20182022/Combined_Flights_2019.parquet', columns=KEEP_COLS)
df_2020 = pd.read_parquet('/kaggle/input/flight-delay-dataset-20182022/Combined_Flights_2020.parquet', columns=KEEP_COLS)
df_2021 = pd.read_parquet('/kaggle/input/flight-delay-dataset-20182022/Combined_Flights_2021.parquet', columns=KEEP_COLS)
df_2022 = pd.read_parquet('/kaggle/input/flight-delay-dataset-20182022/Combined_Flights_2022.parquet', columns=KEEP_COLS)

Now we will need to concatenate the data from different years into a single DataFrame for easier analysis.

In [5]:
df = pd.concat([df_2018, df_2019, df_2020, df_2021, df_2022], ignore_index=True)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29193782 entries, 0 to 29193781
Data columns (total 10 columns):
 #   Column                       Dtype         
---  ------                       -----         
 0   FlightDate                   datetime64[us]
 1   DepTimeBlk                   object        
 2   Origin                       object        
 3   Dest                         object        
 4   IATA_Code_Operating_Airline  object        
 5   DepDelayMinutes              float64       
 6   Distance                     float64       
 7   CRSElapsedTime               float64       
 8   Cancelled                    bool          
 9   ArrTimeBlk                   object        
dtypes: bool(1), datetime64[us](1), float64(3), object(5)
memory usage: 2.0+ GB


In [7]:
df.describe()

,FlightDate,DepDelayMinutes,Distance,CRSElapsedTime
count,29193782,2.843070e+07,2.919378e+07,2.919376e+07
mean,2020-04-23 22:27:03.485606,1.278311e+01,7.797346e+02,1.387605e+02
min,2018-01-01 00:00:00,0.000000e+00,1.600000e+01,-2.920000e+02
25%,2019-03-18 00:00:00,0.000000e+00,3.540000e+02,8.800000e+01
50%,2020-02-08 00:00:00,0.000000e+00,6.260000e+02,1.210000e+02
75%,2021-07-17 00:00:00,5.000000e+00,1.014000e+03,1.690000e+02
max,2022-07-31 00:00:00,7.223000e+03,5.812000e+03,1.645000e+03
std,NaN,4.617337e+01,5.812739e+02,7.077316e+01


In [8]:
# TODO: do not train the model, until you make sure that you can obtain distance and CRSElapsedTime from the API
df.head()

,FlightDate,DepTimeBlk,Origin,Dest,IATA_Code_Operating_Airline,DepDelayMinutes,Distance,CRSElapsedTime,Cancelled,ArrTimeBlk
0,2018-01-23,1200-1259,ABY,ATL,9E,0.0,145.0,62.0,False,1300-1359
1,2018-01-24,1200-1259,ABY,ATL,9E,0.0,145.0,62.0,False,1300-1359
2,2018-01-25,1200-1259,ABY,ATL,9E,0.0,145.0,62.0,False,1300-1359
3,2018-01-26,1200-1259,ABY,ATL,9E,0.0,145.0,62.0,False,1300-1359
4,2018-01-27,1400-1459,ABY,ATL,9E,0.0,145.0,60.0,False,1500-1559


Our goal is to predict delay, so we do not need cancelled flights. We will filter them out during the data loading phase.

In [9]:
df = df[df['Cancelled'] == 0].copy()
df.drop(columns=['Cancelled'], inplace=True)
df.head()

,FlightDate,DepTimeBlk,Origin,Dest,IATA_Code_Operating_Airline,DepDelayMinutes,Distance,CRSElapsedTime,ArrTimeBlk
0,2018-01-23,1200-1259,ABY,ATL,9E,0.0,145.0,62.0,1300-1359
1,2018-01-24,1200-1259,ABY,ATL,9E,0.0,145.0,62.0,1300-1359
2,2018-01-25,1200-1259,ABY,ATL,9E,0.0,145.0,62.0,1300-1359
3,2018-01-26,1200-1259,ABY,ATL,9E,0.0,145.0,62.0,1300-1359
4,2018-01-27,1400-1459,ABY,ATL,9E,0.0,145.0,60.0,1500-1559


In [10]:
df.shape

(28416515, 9)

Let's refine the date & time columns for departure and arrival times to create a proper datetime representation.

In [11]:
df['FlightDate'] = pd.to_datetime(df['FlightDate'])
df['Hour'] = df['DepTimeBlk'].str.slice(0, 2).astype(int)
df.drop(columns=['DepTimeBlk'], inplace=True)
df["datetime"] = (
    df["FlightDate"]
    + pd.to_timedelta(df["Hour"], unit="h")
)
df.head()

,FlightDate,Origin,Dest,IATA_Code_Operating_Airline,DepDelayMinutes,Distance,CRSElapsedTime,ArrTimeBlk,Hour,datetime
0,2018-01-23,ABY,ATL,9E,0.0,145.0,62.0,1300-1359,12,2018-01-23 12:00:00
1,2018-01-24,ABY,ATL,9E,0.0,145.0,62.0,1300-1359,12,2018-01-24 12:00:00
2,2018-01-25,ABY,ATL,9E,0.0,145.0,62.0,1300-1359,12,2018-01-25 12:00:00
3,2018-01-26,ABY,ATL,9E,0.0,145.0,62.0,1300-1359,12,2018-01-26 12:00:00
4,2018-01-27,ABY,ATL,9E,0.0,145.0,60.0,1500-1559,14,2018-01-27 14:00:00


In [12]:
df['Hour_Arrival'] = df['ArrTimeBlk'].str.slice(0, 2).astype(int)
df['arrival_next_day'] = df['Hour_Arrival'] < df['Hour']
df["arr_datetime"] = (
    df["FlightDate"]
    + pd.to_timedelta(df["Hour_Arrival"], unit="h")
    + pd.to_timedelta(df["arrival_next_day"].astype(int), unit="D")
)
df.drop(columns=['ArrTimeBlk', 'Hour_Arrival', 'arrival_next_day', 'Hour', 'FlightDate'], inplace=True)


In [13]:
df.head()

,Origin,Dest,IATA_Code_Operating_Airline,DepDelayMinutes,Distance,CRSElapsedTime,datetime,arr_datetime
0,ABY,ATL,9E,0.0,145.0,62.0,2018-01-23 12:00:00,2018-01-23 13:00:00
1,ABY,ATL,9E,0.0,145.0,62.0,2018-01-24 12:00:00,2018-01-24 13:00:00
2,ABY,ATL,9E,0.0,145.0,62.0,2018-01-25 12:00:00,2018-01-25 13:00:00
3,ABY,ATL,9E,0.0,145.0,62.0,2018-01-26 12:00:00,2018-01-26 13:00:00
4,ABY,ATL,9E,0.0,145.0,60.0,2018-01-27 14:00:00,2018-01-27 15:00:00


In order to calculate the number of flights departing from an airport, and arriving to the same airport, within the same hour, we need to group by the origin and deparuture datetime, and then do the same for arrival. Then we can add these counts as new features to the main dataframe.

First, we will group by 'Origin' and 'datetime' to get the count of departures per hour for each airport.

In [14]:
dep_counts = (
    df.groupby(["Origin", "datetime"])
      .size()
      .reset_index(name="departures_per_hour")
      .rename(columns={"Origin": "airport", "datetime": "hour"})
)

arr_counts = (
    df.groupby(["Dest", "arr_datetime"])
      .size()
      .reset_index(name="arrivals_per_hour")
      .rename(columns={"Dest": "airport", "arr_datetime": "hour"})
)


In [15]:
dep_counts.head()

,airport,hour,departures_per_hour
0,ABE,2018-01-01 06:00:00,2
1,ABE,2018-01-01 09:00:00,2
2,ABE,2018-01-01 17:00:00,1
3,ABE,2018-01-01 20:00:00,1
4,ABE,2018-01-02 06:00:00,3


In [16]:
arr_counts.head()

,airport,hour,arrivals_per_hour
0,ABE,2018-01-01 09:00:00,1
1,ABE,2018-01-01 16:00:00,1
2,ABE,2018-01-01 17:00:00,2
3,ABE,2018-01-01 19:00:00,1
4,ABE,2018-01-01 22:00:00,1


In [17]:
congestion = (
    dep_counts
    .merge(
        arr_counts,
        on=["airport", "hour"],
        how="outer"
    )
    .fillna(0)
)

congestion["scheduled_congestion"] = (
    congestion["departures_per_hour"]
    + congestion["arrivals_per_hour"]
)

congestion.head(10)

,airport,hour,departures_per_hour,arrivals_per_hour,scheduled_congestion
0,ABE,2018-01-01 06:00:00,2.0,0.0,2.0
1,ABE,2018-01-01 09:00:00,2.0,1.0,3.0
2,ABE,2018-01-01 16:00:00,0.0,1.0,1.0
3,ABE,2018-01-01 17:00:00,1.0,2.0,3.0
4,ABE,2018-01-01 19:00:00,0.0,1.0,1.0
5,ABE,2018-01-01 20:00:00,1.0,0.0,1.0
6,ABE,2018-01-01 22:00:00,0.0,1.0,1.0
7,ABE,2018-01-02 06:00:00,3.0,0.0,3.0
8,ABE,2018-01-02 09:00:00,1.0,1.0,2.0
9,ABE,2018-01-02 15:00:00,0.0,1.0,1.0


In [18]:
congestion[congestion['scheduled_congestion'] == 132]

,airport,hour,departures_per_hour,arrivals_per_hour,scheduled_congestion
294598,ATL,2018-01-02 14:00:00,51.0,81.0,132.0
294699,ATL,2018-01-07 20:00:00,48.0,84.0,132.0
294801,ATL,2018-01-13 08:00:00,44.0,88.0,132.0
294820,ATL,2018-01-14 08:00:00,53.0,79.0,132.0
294851,ATL,2018-01-15 20:00:00,42.0,90.0,132.0
...,...,...,...,...,...
4423238,ORD,2022-06-23 18:00:00,53.0,79.0,132.0
4423252,ORD,2022-06-24 13:00:00,71.0,61.0,132.0
4423371,ORD,2022-06-30 18:00:00,49.0,83.0,132.0
4423637,ORD,2022-07-14 18:00:00,54.0,78.0,132.0


Now let's merge these counts back into the main dataframe.

In [19]:
df = df.merge(
    congestion[["airport", "hour", "scheduled_congestion"]],
    left_on=["Origin", "datetime"],
    right_on=["airport", "hour"],
    how="left",
    validate="many_to_one"
)
df

,Origin,Dest,IATA_Code_Operating_Airline,DepDelayMinutes,Distance,CRSElapsedTime,datetime,arr_datetime,airport,hour,scheduled_congestion
0,ABY,ATL,9E,0.0,145.0,62.0,2018-01-23 12:00:00,2018-01-23 13:00:00,ABY,2018-01-23 12:00:00,1.0
1,ABY,ATL,9E,0.0,145.0,62.0,2018-01-24 12:00:00,2018-01-24 13:00:00,ABY,2018-01-24 12:00:00,1.0
2,ABY,ATL,9E,0.0,145.0,62.0,2018-01-25 12:00:00,2018-01-25 13:00:00,ABY,2018-01-25 12:00:00,1.0
3,ABY,ATL,9E,0.0,145.0,62.0,2018-01-26 12:00:00,2018-01-26 13:00:00,ABY,2018-01-26 12:00:00,1.0
4,ABY,ATL,9E,0.0,145.0,60.0,2018-01-27 14:00:00,2018-01-27 15:00:00,ABY,2018-01-27 14:00:00,1.0
...,...,...,...,...,...,...,...,...,...,...,...
28416510,EWR,MEM,YX,154.0,946.0,182.0,2022-03-19 20:00:00,2022-03-19 22:00:00,EWR,2022-03-19 20:00:00,30.0
28416511,MSY,EWR,YX,25.0,1167.0,185.0,2022-03-31 19:00:00,2022-03-31 23:00:00,MSY,2022-03-31 19:00:00,11.0
28416512,ALB,ORD,YX,378.0,723.0,158.0,2022-03-08 17:00:00,2022-03-08 18:00:00,ALB,2022-03-08 17:00:00,6.0
28416513,EWR,PIT,YX,113.0,319.0,86.0,2022-03-25 21:00:00,2022-03-25 22:00:00,EWR,2022-03-25 21:00:00,34.0


In [20]:
df[df['scheduled_congestion'] == 132]

,Origin,Dest,IATA_Code_Operating_Airline,DepDelayMinutes,Distance,CRSElapsedTime,datetime,arr_datetime,airport,hour,scheduled_congestion
563,ATL,FAY,9E,0.0,331.0,76.0,2018-01-13 08:00:00,2018-01-13 09:00:00,ATL,2018-01-13 08:00:00,132.0
564,ATL,FAY,9E,8.0,331.0,78.0,2018-01-14 08:00:00,2018-01-14 09:00:00,ATL,2018-01-14 08:00:00,132.0
576,ATL,FAY,9E,47.0,331.0,76.0,2018-01-27 08:00:00,2018-01-27 09:00:00,ATL,2018-01-27 08:00:00,132.0
657,ATL,FSM,9E,0.0,579.0,123.0,2018-01-02 14:00:00,2018-01-02 15:00:00,ATL,2018-01-02 14:00:00,132.0
1235,ATL,FAY,9E,0.0,331.0,75.0,2018-01-25 13:00:00,2018-01-25 14:00:00,ATL,2018-01-25 13:00:00,132.0
...,...,...,...,...,...,...,...,...,...,...,...
28415352,ORD,MSN,YX,18.0,109.0,57.0,2022-03-28 09:00:00,2022-03-28 10:00:00,ORD,2022-03-28 09:00:00,132.0
28416213,ORD,PIT,YX,9.0,413.0,95.0,2022-03-31 18:00:00,2022-03-31 21:00:00,ORD,2022-03-31 18:00:00,132.0
28416240,ORD,IND,YX,0.0,177.0,72.0,2022-03-31 18:00:00,2022-03-31 20:00:00,ORD,2022-03-31 18:00:00,132.0
28416330,ORD,CMH,YX,0.0,296.0,85.0,2022-03-31 18:00:00,2022-03-31 21:00:00,ORD,2022-03-31 18:00:00,132.0


Now we can include weather data into our dataset. These classes were created in order to simplify the apis calls on the notebook.

In [86]:
from openmeteo_api.src.openmeteoapi.WeatherData import Weather
from openmeteo_api.src.openmeteoapi.APICaller import OpenMeteoAPICaller
import os
from dotenv import load_dotenv

The flight delay dataset is from 2018 to 2022, so all we need to do is getting the weather data for this range of time for each unique airpor, and then join them together.

In [67]:
print(set(df['Origin'].unique()) == set(df['Dest'].unique()))
print(len(set(df['Origin'].unique())))
# print(df[df['Origin']== "ISN"])

True
388


In [77]:
%cat /content/drive/MyDrive/Colab\ Notebooks/FlightDelay/openmeteo_api/src/openmeteoapi/WeatherData.py

"""
Module for fetching and processing weather data from the Open-Meteo API.

Author: Rashed Aldulijan
Created: 2025-12-27

"""
import pandas as pd
from typing import List, Optional, Tuple, Dict, Any
import airportsdata
import math

import os
from dotenv import load_dotenv


load_dotenv()



class Weather:
    
    @staticmethod
    def _haversine_distance_km(
        lat1: float, lon1: float, lat2: float, lon2: float
    ) -> float:
        r = 6371.0088
        phi1 = math.radians(lat1)
        phi2 = math.radians(lat2)
        dphi = math.radians(lat2 - lat1)
        dlambda = math.radians(lon2 - lon1)

        a = (
            math.sin(dphi / 2) ** 2
            + math.cos(phi1) * math.cos(phi2) * math.sin(dlambda / 2) ** 2
        )
        return 2 * r * math.asin(math.sqrt(a))

    @staticmethod
    def find_nearest_airport(
        latitude: float,
        longitude: float,
        code_type: str = "iata",
        max_distance_km: Optional[float] = None,
    ) -> Tuple[str, Di

In [87]:
from tqdm import tqdm
airports = list(df['Origin'].unique())
api_caller_weather = OpenMeteoAPICaller()
weathers = []
start_date = "2018-01-01"
end_date = "2022-12-31"
for i in tqdm(airports):
  weather = Weather(
    api_caller=api_caller_weather,
    airport_code=i,
    code_type="iata",
    start_date=start_date,
    end_date=end_date,
)

 59%|█████▉    | 229/388 [00:17<00:11, 13.44it/s]


ValueError: Airport with ICAO code ISN not found.